# Бэггинг

Пусть $\xi_1, \xi_2, \dots, \xi_n$ - одинаково распределённые величины, скоррелированные с коэффициентом корреляции $\rho$ и дисперсией $\sigma^2$. 

$$ Var ( \bar{\xi} )  =  Var  \left(\frac1n \sum_{i=1}^n \xi_i  \right) = \frac1{n^2} cov (\sum_{i=1}^n \xi_i, \sum_{i=1}^n \xi_i) = \frac1{n^2} \sum_{i=1, j=1}^n cov(\xi_i, \xi_j) =$$ $$ = \frac1{n^2} \sum_{i=1}^n var(\xi_i) + \frac1{n^2} \sum_{i=1, j=1, i\neq j}^n cov (\xi_i, \xi_j) = \frac1{n^2} \sum_{i=1}^n \sigma^2+ \frac1{n^2} \sum_{i=1, j=1, i\neq j}^n \rho \sigma^2 =$$
$$ = \frac1{n^2} n \sigma^2 + \frac1{n^2} n(n-1) \rho \sigma^2  = \frac{\sigma^2( 1 + \rho(n-1))}{n}$$

Таким образом, чем менее скоррелированы между собой величины, тем меньше дисперсия их среднего. 

Грубо говоря в этом и состоит идея бэггинга: 
сделать много максимально независимых моделей, усреднить из и получить более устойчивые предсказания 

# Бэггинг над решающими деревьями

Посмотрим, какие модели можно получить из деревьев с помощью их рандомизации

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('hr_kaggle.csv')

target = 'left'
features = [c for c in data if c != target]

X, y = data[features], data[target]

In [3]:
rnd_d3 = DecisionTreeClassifier(max_features=int(len(features) ** 0.5)) # Решающее дерево с рандомизацией в сплитах
d3 = DecisionTreeClassifier() # Обычное решающее дерево

Качество классификации решающим деревом с настройками по-умолчанию:

In [4]:
print("Decision tree:", cross_val_score(d3, X, y).mean())

Decision tree: 0.65137624858305


Бэггинг над решающими деревьями:

In [5]:
print("D3 bagging:", cross_val_score(BaggingClassifier(d3, random_state=42), X, y).mean())

D3 bagging: 0.7174495299059812


Усредненная модель оказалась намного лучше: 
у решающих деревьев есть существенный недостаток - нестабильность получаемого дерева при небольших изменениях в выборке, но бэггинг обращает этот недостаток в достоинство, ведь усредненная модель работает лучше, когда базовые модели слабо скоррелированы

Изучив параметры DecisionTreeClassifier, можно найти хороший способ сделать деревья еще более различными - при построении каждого узла отбирать случайные max_features признаков и искать информативное разбиение только по одному из них.

In [26]:
print("Randomized decision tree:", cross_val_score(rnd_d3, X, y).mean())

Randomized decision tree: 0.665644448889778


In [30]:
print("Randomized D3 Bagging:", cross_val_score(BaggingClassifier(rnd_d3), X, y).mean())

Randomized D3 Bagging: 0.7190487697539508


В среднем, качество получается еще лучше. Для выбора числа признаков использовалась часто применяемая на практике эвристика - брать корень из общего числа признаков. Если бы мы решали задачу регрессии - брали бы треть от общего числа.

Повторные запуски cross_val_score будут показывать различное качество модели.
Это зависит от параметра рандомизации модели "random_state".

### BaggingClassifier vs RandomForestClassifier():

Bagging improves variance by averaging/majority selection of outcome from multiple fully grown trees on variants of training set. It uses Bootstrap with replacement to generate multiple training sets.

Bagging has a single parameter, which is the number of trees. All trees are fully grown binary tree (unpruned) and at each node in the tree one searches over **all** features to find the feature that best splits the data at that node.

Random Forests improve variance by reducing correlation between trees, this is accomplished by random selection of feature-subset for split at each node. 

Random forests has 2 parameters: the number of trees (the same as bagging) and  $mtry$ (unique to randomforests)   which is how many features to search over to find the best feature. This parameter is usually $1/3*D$ for regression and $\sqrt{D}$ for classification. Thus during tree creation randomly $mtry$ number of features are chosen from all available features and the best feature that splits the data is chosen.




In [15]:
print("Randomized D3 Bagging:", cross_val_score(BaggingClassifier(rnd_d3, random_state=42), X, y).mean())

Randomized D3 Bagging: 0.7194494632259785


In [11]:
print("Random Forest:", cross_val_score(RandomForestClassifier(random_state=42), X, y).mean())

Random Forest: 0.7232495965859839


In [10]:
print("Logistic Regression:", cross_val_score(LogisticRegression(), X, y).mean())

Logistic Regression: 0.6287053143962126
